# Visualize optimized models comparison results

This notebook scans the `optimized_models` directory for `comparison_*.json` files, aggregates the reported metrics (fps, avg_inference_time_ms, model_size_mb, etc.), and renders interactive Plotly charts to compare variants (original, fp16, int8, batch) across models. Run the code cell below to produce the charts.

In [20]:
# Scan optimized_models for comparison_*.json but only use the latest session per model — only direct folders named "seg_*" — aggregate and plot with Plotly
import json
from pathlib import Path
import pandas as pd
import plotly.express as px

# root should be the optimized_models folder (not the repo root)
root = Path("./")

rows = []
# iterate immediate child model folders whose name starts with 'seg_'
model_folders = sorted([p for p in root.iterdir() if p.is_dir() and p.name.startswith('seg_')])
for model_dir in model_folders:
    # find session subfolders inside the model_dir (e.g., 20250910_204734_000015)
    session_dirs = [d for d in model_dir.iterdir() if d.is_dir()]
    if not session_dirs:
        # sometimes files may be directly inside; check for comparison files directly
        candidates = list(model_dir.rglob('comparison_*.json'))
        if not candidates:
            continue
        latest_session_root = model_dir
    else:
        # choose the latest session by folder name (YYYYMMDD... lexicographical order works)
        # choose session by index: 1 = latest, 2 = second latest, etc. (lexicographical order)
        session_dirs_sorted = sorted(session_dirs, key=lambda d: d.name)
        session_index = 1  # 1 = latest, 2 = second latest, etc.
        if len(session_dirs_sorted) < session_index:
            continue  # skip if not enough sessions
        latest_session = session_dirs_sorted[-session_index]
        latest_session_root = latest_session

    # search within the latest session root for comparison_*.json files
    found = list(latest_session_root.rglob('comparison_*.json'))
    if not found:
        # if nothing found, try one level deeper (some layouts have an extra model-named folder)
        deeper = []
        for child in latest_session_root.iterdir():
            if child.is_dir():
                deeper.extend(child.rglob('comparison_*.json'))
        found = deeper

    if not found:
        # nothing to process for this model
        print(f"No comparison_*.json for model folder {model_dir.name} (checked latest session {latest_session_root})")
        continue

    # process all comparison files found in the chosen latest session
    for p in found:
        try:
            data = json.loads(p.read_text(encoding='utf-8'))
        except Exception as e:
            # print(f"Skipping {p}: {e}")
            continue
        model_label = model_dir.name
        # flatten the metrics
        for variant, metrics in data.items():
            row = dict(model=model_label, variant=variant, filepath=str(p), session=str(latest_session_root))
            if not isinstance(metrics, dict):
                # safety: sometimes JSON may be a list/other
                # print(f" Skipping {p} - invalid metrics format for variant {variant}")
                continue
            for k, v in metrics.items():
                row[k] = v
            rows.append(row)

if not rows:
    print('No comparison_*.json files found under', root, 'for latest sessions of seg_* folders')
else:
    df = pd.DataFrame(rows)
    # tidy up model names
    df['model'] = df['model'].astype(str)
    df['batch_size'] = df['batch_size'].fillna(1).astype(int)
    # Remove 'filepath' and 'session' columns for visualization only
    df_vis = df.drop(columns=['filepath', 'session'])
    display(df_vis)

    # Plot: FPS by model and variant
    if 'fps' in df.columns:
        fig1 = px.bar(df, x='model', y='fps', color='variant', barmode='group', title='FPS by model and variant')
        fig1.show()

    # Plot: inference time ms by model and variant
    if 'avg_inference_time_ms' in df.columns:
        fig2 = px.bar(df, x='model', y='avg_inference_time_ms', color='variant', barmode='group', title='Avg inference time (ms) by model and variant')
        fig2.update_yaxes(type='log')
        fig2.show()

    # Scatter: model size vs fps
    if 'model_size_mb' in df.columns and 'fps' in df.columns:
        fig3 = px.scatter(df, x='model_size_mb', y='fps', color='variant', hover_data=['model', 'filepath', 'session'], size='fps', title='Model size (MB) vs FPS')
        fig3.show()

    print('Done.')

,model,variant,avg_inference_time_ms,fps,model_size_mb,batch_size
0,seg_b0_ade,original,89.074898,11.226507,14.329124,1
1,seg_b0_ade,fp16,47.638083,20.991609,7.164581,1
2,seg_b0_ade,int8,81.778049,12.228220,14.329124,1
3,seg_b0_ade,batch,75.335884,13.273887,14.329124,4
4,seg_b0_city,original,86.810684,11.519319,14.200695,1
5,seg_b0_city,fp16,46.374226,21.563702,7.100367,1
6,seg_b0_city,int8,78.359318,12.761724,14.200695,1
7,seg_b0_city,batch,72.506046,13.791953,14.200695,4
8,seg_b1_ade,original,114.189386,8.757381,52.335350,1
9,seg_b1_ade,fp16,55.843925,17.907051,26.167694,1


Done.
